## 准备数据

In [132]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [133]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [134]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.uniform([784, 84], minval=-0.1, maxval=0.1), trainable=True)
        self.b1 = tf.Variable(tf.zeros([84]), trainable=True)
        self.W2 = tf.Variable(tf.random.uniform([84, 10], minval=-0.1, maxval=0.1), trainable=True)
        self.b2 = tf.Variable(tf.zeros([10]), trainable=True)
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, (-1, 784))
        h1 = tf.nn.relu(x @ self.W1 + self.b1)
        h2 = tf.nn.relu(h1 @ self.W2 + self.b2)
        logits = h2

        return logits
        
model = myModel()

optimizer = optimizers.RMSprop(learning_rate=0.001)

## 计算 loss

In [135]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(grads, trainable_vars))
    # for g, v in zip(grads, trainable_vars):
    #     v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [136]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.30336 ; accuracy 0.100033335
epoch 1 : loss 2.2079988 ; accuracy 0.37055
epoch 2 : loss 2.1142883 ; accuracy 0.49508333
epoch 3 : loss 2.0177805 ; accuracy 0.59215
epoch 4 : loss 1.9228511 ; accuracy 0.63418335
epoch 5 : loss 1.8321377 ; accuracy 0.6692
epoch 6 : loss 1.7476119 ; accuracy 0.68761665
epoch 7 : loss 1.6689857 ; accuracy 0.7046
epoch 8 : loss 1.5958017 ; accuracy 0.71486664
epoch 9 : loss 1.5268681 ; accuracy 0.72421664
epoch 10 : loss 1.4608942 ; accuracy 0.73345
epoch 11 : loss 1.397205 ; accuracy 0.742
epoch 12 : loss 1.3364869 ; accuracy 0.7524167
epoch 13 : loss 1.2790956 ; accuracy 0.7632667
epoch 14 : loss 1.2258387 ; accuracy 0.7728
epoch 15 : loss 1.1763937 ; accuracy 0.78001666
epoch 16 : loss 1.1303355 ; accuracy 0.78718334
epoch 17 : loss 1.0873034 ; accuracy 0.7934833
epoch 18 : loss 1.0470201 ; accuracy 0.79981667
epoch 19 : loss 1.0092767 ; accuracy 0.80581665
epoch 20 : loss 0.97396743 ; accuracy 0.81013334
epoch 21 : loss 0.9409327 ; accu

KeyboardInterrupt: 